In [1]:
pip install rdkit

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for rdkit from https://files.pythonhosted.org/packages/bb/ee/e74ab769bb62eb8b3c79556c2af01697800f65c6e137af2f6c5ce58753bd/rdkit-2023.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 117.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import pandas as pd

In [4]:
data = pd.read_csv('../data/opentargets/drugs_smiles.csv')

In [5]:
mols = [Chem.MolFromSmiles(smiles) for smiles in data['smiles']]
fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in mols]

In [6]:
similarities = []
for i in range(len(fps)):
    for j in range(i + 1, len(fps)):
        similarity = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        similarities.append({
            'Drug1': data['drug'].iloc[i],
            'Drug2': data['drug'].iloc[j],
            'Similarity': similarity
        })
similarity_df = pd.DataFrame(similarities)

In [7]:
similarity_df.to_csv('../data/opentargets/drug_similarity.csv', index=False)

protein-protein similarity

In [1]:
pip install networkx

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import networkx as nx
from itertools import combinations

# Load your dataset into a DataFrame (replace 'your_dataset.csv' with the actual file path)
data = pd.read_csv('../data/opentargets/targets_sequences.csv')

G = nx.Graph()

# Create pairwise combinations of protein IDs
protein_ids = data['target'].tolist()
for protein_pair in combinations(protein_ids, 2):
    G.add_edge(protein_pair[0], protein_pair[1])

# Initialize an empty list to store protein pairs
protein_pairs = []

# Use combinations to generate unique pairs
for protein_pair in combinations(protein_ids, 2):
    protein_pairs.append(protein_pair)

In [ ]:
# Create a graph from protein pairs
G = nx.Graph()
G.add_edges_from(protein_pairs)

similarities = []

# Calculate Jaccard similarity for each pair
for i in range(len(protein_pairs)):
    for j in range(i + 1, len(protein_pairs)):
        protein1 = protein_pairs[i][0]
        protein2 = protein_pairs[j][1]
        
        # Calculate Jaccard similarity
        common_neighbors = len(list(nx.common_neighbors(G, protein1, protein2)))
        union_neighbors = len(set(G[protein1]) | set(G[protein2]))
        jaccard_similarity = common_neighbors / union_neighbors
        
        # Append to the list of similarities
        similarities.append({
            'Protein1': protein1,
            'Protein2': protein2,
            'JaccardSimilarity': jaccard_similarity
        })

# Create a DataFrame from the list of similarities
similarity_df = pd.DataFrame(similarities)

In [4]:
similarity_df.to_csv('../data/opentargets/protein_similarity.csv', index=False)